In [38]:
# essas libs sao usadas para: 
import pandas as pd # manipulacao de dados
import yfinance as yf # coleta de dados

In [39]:
file_name = "carteira.txt" # nome do arquivo

In [40]:
carteira = pd.read_csv(file_name, sep=' - ', header=None, names=['Ticker', 'Quantidade'], engine='python') # leitura do arquivo acima

In [41]:
# removendo espaços em branco
carteira['Ticker'] = carteira['Ticker'].str.strip() # remove espaços em branco
carteira['Quantidade'] = carteira['Quantidade'].astype(int) # converte para inteiro

In [42]:
def buscar_preco(ticker): # função para buscar o preço do ativo
    try:
        return yf.Ticker(ticker + ".SA").history(period="1d")['Close'].iloc[-1] # retorna o preço do ativo
    except:
        return None # retorna None caso não encontre o ativo

In [43]:
carteira['Preço Atual'] = carteira['Ticker'].apply(buscar_preco) # adiciona a coluna Preço Atual na carteira

In [44]:
carteira['Valor Total'] = carteira['Quantidade'] * carteira['Preço Atual'] # adiciona a coluna Valor Total na carteira

In [45]:
valor_inicial = (carteira['Quantidade'] * carteira['Preço Atual']).sum() # calcula o valor inicial da carteira
valor_atual = carteira['Valor Total'].sum() # calcula o valor atual da carteira

In [46]:
df_historico = pd.concat([yf.Ticker(ticker + ".SA").history(period="1d") for ticker in carteira['Ticker']]) # pega o histórico de cada ativo
rentabilidade_ibov = ((df_historico['Close'].iloc[-1] - df_historico['Open'].iloc[0]) / df_historico['Open'].iloc[0]) * 100 # calcula a rentabilidade do ibov
rentabilidade = ((valor_atual - valor_inicial) / valor_inicial) * 100 # calcula a rentabilidade da carteira

In [47]:
carteira['Contribuição (%)'] = ((carteira['Valor Total'] - (valor_inicial / len(carteira))) / valor_inicial) * 100 # calcula a contribuição de cada ativo

In [48]:
# printa os resultados
print(f"Valor Inicial da Carteira: {valor_inicial:.6f}")
print(f"Valor Atual da Carteira: {valor_atual:.6f}")
print(f"Rentabilidade da Carteira: {rentabilidade:.1f}%")
print(f"Rentabilidade do Índice Bovespa: {rentabilidade_ibov:.1f}%")
print("\nContribuição dos ativos para a rentabilidade total (%):")
print(carteira[['Ticker', 'Contribuição (%)']].set_index('Ticker'))

Valor Inicial da Carteira: 2919765.964317
Valor Atual da Carteira: 2919765.964317
Rentabilidade da Carteira: 0.0%
Rentabilidade do Índice Bovespa: -46.9%

Contribuição dos ativos para a rentabilidade total (%):
        Contribuição (%)
Ticker                  
ITUB4         -18.842030
BBAS3          75.555603
VALE3         -19.260078
EGIE3         -17.514184
SLCE3         -19.939310


In [ ]:
df_historico = pd.concat([yf.Ticker(ticker + ".SA").history(period="1y")['Close'].rename(ticker) for ticker in carteira['Ticker']], axis=1) # pega o histórico de cada ativo
# obs: as colunas são coletadas com base na variavel ticker e passado para a função history que por sua vez pega o historico de cada ativo e as organiza em colunas atravez do metodo rename

df_historico.index = df_historico.index.strftime('%d-%m-%Y') # formata a data

display(df_historico.tail(5).style.set_caption("Histórico de Preços dos Ativos e do IBOV") # printa o histórico dos ativos dos ultimos 5 dias
       .format("{:.2f}") # formata os valores
       .set_properties(**{'background-color': '#121212', 'color': 'white', 'border-color': 'white'})) # formata o estilo

,ITUB4,BBAS3,VALE3,EGIE3,SLCE3
Date,,,,,
29-01-2025,33.20,27.44,52.80,36.72,17.52
30-01-2025,34.01,27.64,55.03,36.85,17.89
31-01-2025,33.79,27.68,54.17,36.80,17.36
03-02-2025,33.48,27.61,54.21,36.31,17.48
04-02-2025,33.79,27.89,54.01,36.29,17.72


In [50]:
print(carteira) # imprime a carteira

  Ticker  Quantidade  Preço Atual   Valor Total  Contribuição (%)
0  ITUB4        1000    33.810001  3.381000e+04        -18.842030
1  BBAS3      100000    27.900000  2.790000e+06         75.555603
2  VALE3         400    54.009998  2.160400e+04        -19.260078
3  EGIE3        2000    36.290001  7.258000e+04        -17.514184
4  SLCE3         100    17.719999  1.772000e+03        -19.939310


In [51]:
carteira.to_csv("resultado_carteira.csv", index=False) # salva a carteira em um arquivo csv